In [1]:
import numpy as np
import datetime

In [2]:
x_train = np.load('data/train/x_train.npy')
# x_test = np.load('data/train/x_test.npy')
y_train = np.load('data/train/y_train.npy')
# y_test = np.load('data/train/y_test.npy')

In [ ]:
from keras import backend as K
from resnet3d import Resnet3DBuilder 
# pip install git+https://github.com/JihongJu/keras-resnet3d.git

bchs = 1
epch = 75

ir = x_train.shape[1] # rows
ic = x_train.shape[2] # cols
ih = x_train.shape[3] # slices

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, ir, ic, ih)
    # x_test = x_test.reshape(x_test.shape[0], 1, ir, ic, ih)
    input_shape = (1, ir, ic, ih)
else:
    x_train = x_train.reshape(x_train.shape[0], ir, ic, ih, 1)
    # x_test = x_test.reshape(x_test.shape[0], ir, ic, ih, 1)
    input_shape = (ir, ic, ih, 1)

nb_classes = y_train.shape[1]
    
model = Resnet3DBuilder.build_resnet_50(input_shape, nb_classes)
model.compile(optimizer = 'adam',
              loss = 'mean_squared_error', 
              metrics = ['mae'])

trainstarttime = datetime.datetime.today()
print("")
print("Train start time: " + str(trainstarttime))

history = model.fit(x_train, y_train,
            batch_size = bchs,
            epochs = epch,
            verbose = 1)
            # validation_data = (x_test, y_test))

trainfinishtime = datetime.datetime.today()
print("Train finish time: " + str(trainfinishtime))
ttime = str(trainfinishtime - trainstarttime)
print("Training time: " + ttime)
print("")

In [ ]:
file = 'data/train/models/resnet_model'

json_string = model.to_json()
fj = file + '.json'
fh = file + '.h5'
open(fj, 'w').write(json_string)
model.save_weights(fh)

print("Model saved.")